## Get images from Landsat 8

### Summary of Workflow to get images for a single location:
1. Run the first cell below if ee is not installed in the jupyter environment.
2. Get lat/long for a specific location from the emissions file https://github.com/avaimar/urban_emissions/blob/main/01_Data/01_Carbon_emissions/AirNow/World_cities_2020_avg_latlon.csv and run the get_box function with that (lat, long).
4. Copy paste the printed variables ('rectangle' and 'region') into the next cell and run. Download images by clicking on the links in the cell output (or they are saved to Drive)

### Notes / Next Steps:
1. Each link downloads a zip file with three TIFF files where each of these corresponds to one of the bands. I am not sure how to visualize this to make sure that the images are correct.
2. I'm using this Landsat dataset hopefully that is correct? https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T2
3. One thing to note is that different locations have different amounts of data available. For example, I had to increase the box size to around 300 - 325km to get any data for London which seems like a huge area. Using this same grid size for the Boston location, for example, returns many more images than is found for London. I average all images in a unit geographic grid cell anyway, but just to note.
4. I don't have enough space on my laptop to download all of this data. I haven't gotten the upload to Google Drive part to work that you had Nicolas...It creates an empty folder in my Drive account but the images are not actually uploaded. Maybe you can help with this.
5. Another next step is to automate this, although it should be fairly simple to pull lat/long from Andrea's CSV file and plug it into these functions. I just want to figure out the Google Drive part first because my laptop can't store all of those files...
6. Finally, should convert these to numpy array format and then we can save them in .npy instead of TIFF format.

## Step 1: Install package if not already

In [2]:
pip install earthengine-api

  Using cached earthengine-api-0.1.248.tar.gz (150 kB)
  Using cached future-0.18.2.tar.gz (829 kB)
  Using cached google_cloud_storage-1.35.0-py2.py3-none-any.whl (96 kB)
  Using cached google_api_python_client-1.12.8-py2.py3-none-any.whl (61 kB)
  Using cached google_auth-1.24.0-py2.py3-none-any.whl (114 kB)
  Using cached google_auth_httplib2-0.0.4-py2.py3-none-any.whl (9.1 kB)
  Using cached httplib2-0.18.1-py3-none-any.whl (95 kB)
  Using cached httplib2shim-0.0.3.tar.gz (17 kB)
  Using cached google_resumable_media-1.2.0-py2.py3-none-any.whl (75 kB)
  Using cached google_cloud_core-1.5.0-py2.py3-none-any.whl (27 kB)
  Using cached google_api_core-1.25.1-py2.py3-none-any.whl (92 kB)
  Using cached uritemplate-3.0.1-py2.py3-none-any.whl (15 kB)
  Using cached rsa-4.7-py3-none-any.whl (34 kB)
  Using cached cachetools-4.2.1-py3-none-any.whl (12 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached protobuf-3.14.0-py2.py3-none-any.whl (173 kB)
  Using c

## Step 2+3: Run the get_box() function with a latitude and longitude point

In [48]:
###### Get Image Bounding Box ######
# Source: adapted from https://github.com/Yichabod/natural_disaster_pred/blob/master/cropping_coordinates.py

import math

# Distances are measured in kilometers.
# Longitudes and latitudes are measured in degrees.
# Earth is assumed to be perfectly spherical.

earth_radius = 6271.0
degrees_to_radians = math.pi/180.0
radians_to_degrees = 180.0/math.pi

def change_in_latitude(kms):
    "Given a distance north, return the change in latitude."
    return (kms/earth_radius)*radians_to_degrees

def change_in_longitude(latitude, kms):
    "Given a latitude and a distance west, return the change in longitude."
    # Find the radius of a circle around the earth at given latitude.
    r = earth_radius*math.cos(latitude*degrees_to_radians)
    return (kms/r)*radians_to_degrees

def ten_km_square(latitude, longitude):
    n = 325 # 3.75
    slat, nlat = latitude+change_in_latitude(-1*n), latitude+change_in_latitude(n)
    wlon = longitude+change_in_longitude(latitude,-1*n)

    elon = longitude+change_in_longitude(latitude, n)
    return(nlat, wlon, slat, elon)

def get_box(lon, lat):
    '''First argument degrees longitude (E is positive, W negative)
        of the landslide location,
        second argument latitude (N positive, S negative),
        in decimal format(not minutes etc.)'''
    nlat, wlon, slat, elon = ten_km_square(lat,lon)
    print("region = '[[{:.4f},{:.4f}], [{:.4f},{:.4f}], [{:.4f},{:.4f}], [{:.4f},{:.4f}]]'".format(wlon,nlat,elon,nlat,wlon,slat,elon,slat))
    print("rectangle = [{:.4f},{:.4f},{:.4f},{:.4f}]".format(wlon,slat,elon,nlat))

In [52]:
# london = (51.5073509,-0.1277583)
# menlo park = (-122.2036486, 37.4237011)
lat, long = 51.5073509, -0.1277583
get_box(lat, long)

var region = '[[48.5379,2.8416], [54.4768,2.8416], [48.5379,-3.0972], [54.4768,-3.0972]]';
var rectangle = [48.5379,-3.0972,54.4768,2.8416];


## Step 4: Copy paste the region and rectange variables here and run cell
Then click the URLs to download the images OR save to drive.

In [61]:
import ee

ee.Initialize()
ee.Authenticate()

startDate = '2020-01-01'
endDate = '2020-12-01'

# London (n=325)
region = '[[48.5379,2.8416], [54.4768,2.8416], [48.5379,-3.0972], [54.4768,-3.0972]]'
rectangle = [48.5379,-3.0972,54.4768,2.8416]

# boston (n=325)
# region = '[[32.9542,-68.0490], [51.2126,-68.0490], [32.9542,-73.9878], [51.2126,-73.9878]]'
# rectangle = [32.9542,-73.9878,51.2126,-68.0490]

bands = ["B2","B3","B4"]

rectangle1 = ee.Geometry.Rectangle(rectangle)

dataset = ee.ImageCollection("LANDSAT/LC08/C01/T2")
dataset = dataset.filterBounds(rectangle1) # Filter region
dataset = dataset.filterDate(startDate, endDate).sort('system:time_start', True) # filter date
dataset = dataset.select(bands) # select RGB channels

count = dataset.size().getInfo()
print('num_images = ', count)
data = dataset.toList(count)

if (count == 0):
    print('No data found.')

else:
    print('Data found')
    dataset_avg = dataset.mean() # get the average over all images in the grid cell
    image = ee.Image(dataset_avg) # convert to an Image
    
    # Option 1: Get clickable download link
    print(image.getDownloadURL({'region': region})) # get link to download
    
    # Option 2: Save to Drive
    # task=ee.batch.Export.image.toDrive(image=image, folder="Carbon_emissions_foo", 
    #                  description='LANDSTAT_{}_{}_{}'.format(lat, long, i), 
    #                  region=region,fileFormat='TFRecord')
    # task.start()
    
    '''
    # To get each image in the collection individually
    for i in range(count):
        image = ee.Image(data.get(i))
        image = image.select(bands)
        print('image number ', i)
        # Option 1: Get clickable download link
        print(image.getDownloadURL({'region': region})) # ,'scale': 10}))
        
        # Option 2: Save to Drive
        # task=ee.batch.Export.image.toDrive(image=image, folder="Carbon_emissions_foo", 
        #                 description='LANDSTAT_{}_{}_{}'.format(lat, long, i), 
        #                 region=region,fileFormat='TFRecord')
        # task.start()
    '''

Enter verification code: 4/1AY0e-g5zQ-S7z3BNgbSvLUmcIG330GXAqYdgEXoZgZQLhkq283Txv2cR3Xw

Successfully saved authorization token.
num_images =  21
Data found
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/4007dcbe784c18e1511d928c3a2b7856-81400cd48163d22baf0b0d07c7da1f05:getPixels
